# QA over SQL data

## Intro
* We will create Q&A app over tabular data in databases.
* These app will allow us to **ask question about data in database and get back natural language answer**.
* **Building Q&A systems of SQL databases requires executing model-generated SQL queries. There are inherent risks in doing this**. Make sure that your database connection permissions are always scoped as narrowly as possible for your chain's needs.

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

## Connect with LLM

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [6]:
#!pip install langchain-community

## Connect with database
* In this example we will use SQLite connection with `street_tree_db` database we have in `data` folder.
* `data` folder must be in root directory of this app.
* We can interface with database using the SQLAlchemy-driven `SQLDatabase` class.

In [3]:
from langchain_community.utilities import SQLDatabase

sqlite_db_path = "data/street_tree_db.sqlite"

db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")

We can create simple chain that takes question and does the following:
* Convert question into SQL query.
* Execute query.
* Use result to answer original question.

The first step in SQL chain is to take user input and convert it to SQL query. LangChain comes with **built-in chain** for this, `create_sql_query_chain`.

In [4]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)

response = chain.invoke({"question": "How many species of trees are in San Francisco?"})

response

'SELECT COUNT(DISTINCT "qSpecies") AS "TotalSpecies" FROM street_trees;'

* We can execute query to make sure it's valid

In [5]:
db.run(response)

'[(148,)]'

* We can also inspect chain directly for its prompts

In [6]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

* Now that we've generated SQL query, we'll want to execute it.
* **This is the most dangerous part of creating SQL chain**. Consider carefully if it is OK to run automated queries over your data. Minimize the database connection permissions as much as possible.
* **Consider adding human approval step to your chains before query execution**.
* We can use QuerySQLDatabaseTool to easily add query execution to our chain.

In [7]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)

write_query = create_sql_query_chain(llm, db)

chain = write_query | execute_query

chain.invoke({"question": "How many species of trees are in San Francisco?"})

'[("Arbutus \'Marina\' :: Hybrid Strawberry Tree",), (\'Afrocarpus gracilior :: Fern Pine\',), ("Thuja occidentalis \'Emerald\' :: Emerald Arborvitae",), ("Magnolia grandiflora \'Little Gem\' :: Little Gem Magnolia",), (\'Platanus x hispanica :: Sycamore: London Plane\',), (\'Ulmus parvifolia :: Chinese Elm\',), (\'Leptospermum laevigatum :: Australian Tea Tree\',), ("Olea europaea \'Fruitless\' :: Fruitless Olive",), (\'Ficus rubiginosa :: Port Jackson Fig\',), (\'Cycas revoluta :: Sago palm\',), (\'Lophostemon confertus :: Brisbane Box\',), (\'Podocarpus gracilor :: Fern Pine\',), (\'Eriobotrya deflexa :: Bronze Loquat\',), (\'Ligustrum lucidum :: Glossy Privet\',), (\'Syagrus romanzoffianum :: Queen Palm\',), ("Ficus microcarpa nitida \'Green Gem\' :: Indian Laurel Fig Tree \'Green Gem\'",), (\'Prunus cerasifera :: Cherry Plum\',), (\':: To Be Determine\',), (\'Myoporum laetum :: Myoporum\',), (\'Tristaniopsis laurina :: Swamp Myrtle\',), (\'Ginkgo biloba :: Maidenhair Tree\',), (\'

* Now that we've got way to automatically generate and execute queries, we just need to **combine original question and SQL query result to generate final answer**.
* We can do this by passing question and result to LLM once more.

In [8]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "How many species of trees are in San Francisco?"})

'There are 148 species of trees in San Francisco.'

* Let's review what is happening in above LCEL:
    * After first RunnablePassthrough.assign, we have runnable with two elements:
{"question": question, "query": write_query.invoke(question)}
    * write_query will generate SQL query in service of answering question.
    * After second RunnablePassthrough.assign, we have added third element "result" that contains execute_query.invoke(query), where query was computed in previous step.
    * These three inputs are formatted into prompt and passed into LLM.
    * The StrOutputParser() plucks out string content of output message.